In [1]:
import os
import torch


fp = "/playpen-ssd/levi/w4c/w4c-25/__exps__/2025-11-12_22-20-32_SaTformer-cat-loss-weightedCCE-lr=1e-5-BS=128-N=64-ATTN=ST^2/best.pth"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12356"
torch.distributed.init_process_group(backend="gloo", rank=0, world_size=1)
wrapper    = torch.load(fp, map_location='cpu', weights_only=False).cpu()
wrapper_m  = wrapper.module.cpu()
state_dict = wrapper.module.state_dict()

[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


In [1]:
import torch
from src.model.SaTformer.SaTformer import SaTformer


model = SaTformer(
    dim=512,
    num_frames=4,       # number HRIT input frames
    num_classes=64,     # number precipitation bins to use
    image_size=32,      # HRIT input spatial dimensions
    patch_size=4,
    channels=11,        # number HRIT radiance channels
    depth=12,           # number transformer encoder blocks
    heads=8,
    dim_head=64,
    attn_dropout=0.1,
    ff_dropout=0.1,
    rotary_emb=False,   # i.e., use postitional embeds
    attn="ST^2"
)

WEIGHTS_FP = "weights/sf-64-cls.pt"

model.load_state_dict(torch.load(WEIGHTS_FP, weights_only=True), strict=False);
model.eval()

# predict using a randomly generated HRIT sequence
HRIT_dummy_input = torch.rand(1, 4, 11, 32, 32)
with torch.no_grad():
    y = model(HRIT_dummy_input)
    print(y.shape)

torch.Size([1, 64])
